In [11]:
#!pip install -U ipython
#!pip install py2neo
#!pip install pandas

Requirement already up-to-date: ipython in /usr/local/lib/python3.6/dist-packages (7.14.0)
  Created wheel for pd: filename=pd-0.0.1-cp36-none-any.whl size=6507 sha256=4f7e87c9a12f67eafd6262fc1478bb3520823eec6092d4f87710499ada3343e8
  Stored in directory: /root/.cache/pip/wheels/b7/26/e7/cda8e7140160193c00ca36c0d7d34bf1289ab3b00fd1708266
Successfully built pd


In [1]:
import py2neo
import pandas as pd

In [2]:
from py2neo import Database,Graph,Node,Relationship

In [3]:
db = Database()
db

<Database uri='bolt://localhost:7687' secure=False user_agent='py2neo/4.3.0 neobolt/1.7.17 Python/3.7.4-final-0 (win32)'>

In [10]:
ml = Graph("bolt://localhost:7687",user="neo4j2",password="neo4j")
start=ml.begin()

In [11]:
df_peliculas=pd.read_csv('./movies_enriched.csv', sep=',')
df_peliculas

,Unnamed: 0,movieId,title,dbpedia_link,genres,director_uri,director,country_uri,country,runtime,writer_uri,writer,starring,languagues,studios
0,0,781,Stealing Beauty (1996),http://dbpedia.org/resource/Stealing_Beauty,Drama,http://dbpedia.org/resource/Bernardo_Bertolucci,Bernardo Bertolucci,http://dbpedia.org/resource/France,France,7140.0,http://dbpedia.org/resource/Susan_Minot,Susan Minot,Sinéad Cusack|Joseph Fiennes|Rachel Weisz|Jere...,French language|German language|Italian langua...,Recorded Picture Company|Fox Searchlight Pictu...
1,1,1799,Suicide Kings (1997),http://dbpedia.org/resource/Suicide_Kings,Comedy|Crime|Drama|Mystery|Thriller,http://dbpedia.org/resource/Peter_O'Fallon,Peter O'Fallon,NaN,NaN,6360.0,NaN,NaN,Henry Thomas|Denis Leary|Christopher Walken|Ja...,NaN,NaN
2,2,521,Romeo Is Bleeding (1993),http://dbpedia.org/resource/Romeo_Is_Bleeding,Crime|Thriller,http://dbpedia.org/resource/Peter_Medak,Peter Medak,NaN,NaN,6600.0,http://dbpedia.org/resource/Hilary_Henkin,Hilary Henkin,Roy Scheider|Annabella Sciorra|Lena Olin|Gary ...,NaN,PolyGram Filmed Entertainment|Working Title Films
3,3,3596,Screwed (2000),http://dbpedia.org/resource/Screwed_(2000_film),Comedy,http://dbpedia.org/resource/Scott_Alexander_an...,Scott Alexander and Larry Karaszewski,NaN,NaN,4860.0,NaN,NaN,Sarah Silverman|Danny DeVito|Norm Macdonald|El...,NaN,Brillstein-Grey Entertainment
4,4,3682,Magnum Force (1973),http://dbpedia.org/resource/Magnum_Force,Action|Crime|Drama|Thriller,http://dbpedia.org/resource/Clint_Eastwood,Clint Eastwood,NaN,NaN,7440.0,http://dbpedia.org/resource/Michael_Cimino,Michael Cimino,Clint Eastwood|Robert Urich|David Soul|Tim Mat...,NaN,The Malpaso Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282,3282,1013,"Parent Trap, The (1961)",http://dbpedia.org/resource/The_Parent_Trap_(1...,Children|Comedy|Romance,http://dbpedia.org/resource/David_Swift_(direc...,David Swift (director),NaN,NaN,7680.0,http://dbpedia.org/resource/David_Swift_(direc...,David Swift (director),Hayley Mills|Brian Keith|Maureen O'Hara,NaN,Walt Disney Pictures
3283,3283,2634,"Mummy, The (1959)",http://dbpedia.org/resource/The_Mummy_(1959_film),Horror,http://dbpedia.org/resource/Terence_Fisher,Terence Fisher,NaN,NaN,5280.0,http://dbpedia.org/resource/Jimmy_Sangster,Jimmy Sangster,Christopher Lee|George Pastell|Peter Cushing|E...,NaN,Hammer Film Productions
3284,3284,2242,"Grandview, U.S.A. (1984)","http://dbpedia.org/resource/Grandview,_U.S.A.",Drama,http://dbpedia.org/resource/Randal_Kleiser,Randal Kleiser,NaN,NaN,5820.0,http://dbpedia.org/resource/Ken_Hixon,Ken Hixon,Troy Donahue|Jamie Lee Curtis|C. Thomas Howell...,NaN,CBS Theatrical Films
3285,3285,3941,Sorority House Massacre (1986),http://dbpedia.org/resource/Sorority_House_Mas...,Horror,NaN,NaN,http://dbpedia.org/resource/United_States,United States,4560.0,NaN,NaN,NaN,NaN,NaN


In [12]:
m=Node("Movie",movieId=df_peliculas["movieId"].values[0],title=df_peliculas["title"].values[0],dbpediaLink=df_peliculas["dbpedia_link"].values[0],genres=df_peliculas["genres"].values[0],country=df_peliculas["country"].values[0])

In [13]:
 ml.run("MATCH (m:Movie) RETURN m").to_table()

m
"(_0:Movie {country: 'France', countryUri: 'http://dbpedia.org/resource/France', dbpediaLink: 'http://dbpedia.org/resource/Stealing_Beauty', director: 'Bernardo Bertolucci', directorUri: 'http://dbpedia.org/resource/Bernardo_Bertolucci', genres: 'Drama', languages: 'French language|German language|Italian language|Spanish language|English language', movieId: '781', runtime: '7140.0', starring: 'Sin\u00e9ad Cusack|Joseph Fiennes|Rachel Weisz|Jeremy Irons|Liv Tyler', studios: 'Recorded Picture Company|Fox Searchlight Pictures|UGC (cinema operator)', title: 'Stealing Beauty (1996)', writeUri: 'http://dbpedia.org/resource/Susan_Minot', writer: 'Susan Minot'})"
"(_3336:Movie {dbpediaLink: 'http://dbpedia.org/resource/Suicide_Kings', director: ""Peter O'Fallon"", directorUri: ""http://dbpedia.org/resource/Peter_O'Fallon"", genres: 'Comedy|Crime|Drama|Mystery|Thriller', movieId: '1799', runtime: '6360.0', starring: 'Henry Thomas|Denis Leary|Christopher Walken|Jay Mohr|Sean Patrick Flanery|Johnny Galecki', title: 'Suicide Kings (1997)'})"
"(_3337:Movie {dbpediaLink: 'http://dbpedia.org/resource/Romeo_Is_Bleeding', director: 'Peter Medak', directorUri: 'http://dbpedia.org/resource/Peter_Medak', genres: 'Crime|Thriller', movieId: '521', runtime: '6600.0', starring: 'Roy Scheider|Annabella Sciorra|Lena Olin|Gary Oldman|Juliette Lewis', studios: 'PolyGram Filmed Entertainment|Working Title Films', title: 'Romeo Is Bleeding (1993)', writeUri: 'http://dbpedia.org/resource/Hilary_Henkin', writer: 'Hilary Henkin'})"
"(_3338:Movie {dbpediaLink: 'http://dbpedia.org/resource/Screwed_(2000_film)', director: 'Scott Alexander and Larry Karaszewski', directorUri: 'http://dbpedia.org/resource/Scott_Alexander_and_Larry_Karaszewski', genres: 'Comedy', movieId: '3596', runtime: '4860.0', starring: 'Sarah Silverman|Danny DeVito|Norm Macdonald|Elaine Stritch|Sherman Hemsley|Daniel Benzali|Dave Chappelle', studios: 'Brillstein-Grey Entertainment', title: 'Screwed (2000)'})"
"(_3339:Movie {dbpediaLink: 'http://dbpedia.org/resource/Magnum_Force', director: 'Clint Eastwood', directorUri: 'http://dbpedia.org/resource/Clint_Eastwood', genres: 'Action|Crime|Drama|Thriller', movieId: '3682', runtime: '7440.0', starring: 'Clint Eastwood|Robert Urich|David Soul|Tim Matheson|Hal Holbrook|Felton Perry|Mitchell Ryan', studios: 'The Malpaso Company', title: 'Magnum Force (1973)', writeUri: 'http://dbpedia.org/resource/Michael_Cimino', writer: 'Michael Cimino'})"
"(_3340:Movie {dbpediaLink: ""http://dbpedia.org/resource/The_Mummy's_Curse"", director: 'Leslie Goodwins', directorUri: 'http://dbpedia.org/resource/Leslie_Goodwins', genres: 'Horror', movieId: '2635', runtime: '3720.0', starring: 'Lon Chaney, Jr.|Virginia Christine', title: ""Mummy's Curse, The (1944)"", writeUri: 'http://dbpedia.org/resource/Leon_Abrams', writer: 'Leon Abrams'})"
"(_3341:Movie {dbpediaLink: 'http://dbpedia.org/resource/Event_Horizon_(film)', director: 'Paul W. S. Anderson', directorUri: 'http://dbpedia.org/resource/Paul_W._S._Anderson', genres: 'Horror|Sci-Fi|Thriller', movieId: '1590', runtime: '5700.0', starring: 'Kathleen Quinlan|Sam Neill|Joely Richardson|Laurence Fishburne', studios: 'Paramount Pictures', title: 'Event Horizon (1997)', writeUri: 'http://dbpedia.org/resource/Andrew_Kevin_Walker', writer: 'Andrew Kevin Walker'})"
"(_3342:Movie {dbpediaLink: 'http://dbpedia.org/resource/Anne_Frank_Remembered', director: 'Jon Blair', directorUri: 'http://dbpedia.org/resource/Jon_Blair', genres: 'Documentary', movieId: '116', runtime: '7020.0', starring: 'Glenn Close|Kenneth Branagh', title: 'Anne Frank Remembered (1995)'})"
"(_3343:Movie {dbpediaLink: ""http://dbpedia.org/resource/Love's_Labour's_Lost_(2000_film)"", genres: 'Comedy|Romance', movieId: '3719', title: ""Love's Labour's Lost (2000)""})"
"(_3344:Movie {dbpediaLink: 'http://dbpedia.org/resource/Breaker_Morant_(film)', director: 'Bruce Beresford', directorUri: 'http://dbpedia.org/resource/Bruce_Beresford', genres: 'Drama|War', m

In [14]:
len(ml.nodes)

3287